### Aggregating synthpop synthetic populations to tract level
1. read in individual csv files containing synthetic populations
2. combine into one large dataframe
3. group by tract to aggregate counts
4. compute new implied variable
5. export as csv to join to tract boundaries in gis

In [82]:
import pandas as pd
import csv
import numpy as np
# library for path names
import glob 
from functools import reduce
pd.options.display.max_columns = 40

In [2]:
# function to compiling dataframe from folder of individual csv files
def compile_df(pathfile_name):
    li = []
    all_files = glob.glob(pathfile_name)
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0, converters={'tract': lambda x: str(x)})
        li.append(df)
    return(pd.concat(li, axis=0, ignore_index=True))

In [4]:
## creating household level variables/tables
households_meck_comp = compile_df("data_outputs/20190323_run/*households.csv")
households_meck_comp.to_csv("data_outputs/20190323_compiled/households_meck.csv")

# converting columns to correct data type
households_meck = pd.read_csv("data_outputs/20190323_compiled/households_meck.csv", 
                       header=0, dtype={'puma00':str, 'puma10':str, 'NP':int,
                                        'TYPE':int,'race_of_head':str,
                                        'hispanic_head':str, 'age_of_head':int,
                                        'hh_age_of_head':str, 'hh_cars':str,
       'hh_children':str, 'hh_income':str, 'hh_race_of_head':str, 'hh_size':str, 'hh_workers':str,
       'seniors':str, 'sf_detached':str, 'tenure_mover':str,'state':str, 'county':str,
       'tract':str, 'block group':str  
                       })

In [66]:
## dropping variables we are not using
households_var = households_meck.drop(columns=['Unnamed: 0.1', 'serialno', 'RT', 'puma00', 'puma10',
       'NP', 'TYPE', 'BLD', 'TEN', 'VEH', 'HINCP', 'MV', 'R18', 'R65',
       'race_of_head', 'hispanic_head', 'age_of_head', 'workers',
       'hh_age_of_head', 'hh_children', 'hh_race_of_head',
       'hh_size', 'hh_workers', 'seniors', 'sf_detached', 'tenure_mover',
       'cat_id', 'state', 'county', 'block group'])

## aggregating households by cars and tract
households_meck_tract_cars = households_var.groupby(["tract","hh_cars"]).count()
households_meck_tract_income = households_var.groupby(["tract","hh_income"]).count()

# resetting index and unstacking to make a wide data frame
house_cars = households_meck_tract_cars.reset_index().pivot(index='tract', columns='hh_cars', values ='Unnamed: 0').fillna(0)
house_income = households_meck_tract_income.reset_index().pivot(index='tract', columns='hh_income', values ='Unnamed: 0').fillna(0) 

households_merged = house_cars.merge(house_income,how='outer', on="tract")

##adding prefix to columns
households_merged.columns = ['s_' + str(col) for col in households_merged.columns]



In [100]:
# person level data
persons_meck_comp = compile_df("data_outputs/20190323_run/*persons.csv")
persons_meck_comp.to_csv("data_outputs/20190323_compiled/persons_meck.csv")
persons_meck = pd.read_csv("data_outputs/20190323_compiled/persons_meck.csv",
                                 header=0, dtype = {'AGEP':int,
       'RELP':int, 'SEX':str,'HISP':str, 'RAC1P':str,
       'hispanic':str, 'person_age':str, 'person_sex':str, 'race':str,
       'state':str, 'county':str, 'tract':str, 'block group':str})

# # aggregating for total pop person level data
persons_var = persons_meck.drop(columns=['Unnamed: 0.1', 'serialno', 'SPORDER', 'puma00', 'puma10',
       'AGEP', 'JWTR', 'RELP', 'SCH', 'SCHL', 'SEX', 'WKHP', 'ESR', 'HISP',
       'PERNP', 'RAC1P',
       'cat_id', 'hh_id', 'state', 'county', 'block group'])
persons_tract_age = persons_var.groupby(["tract","person_age"]).count()
persons_tract_sex = persons_var.groupby(["tract","person_sex"]).count()
persons_tract_race = persons_var.groupby(["tract","race"]).count()
persons_tract_hisp = persons_var.groupby(["tract","hispanic"]).count()

## resetting index and unstacking to make a wide data frame
persons_tract_age = persons_tract_age.reset_index().pivot(index='tract',columns="person_age", values="Unnamed: 0").dropna(0)
persons_tract_sex = persons_tract_sex.reset_index().pivot(index='tract',columns="person_sex", values="Unnamed: 0").dropna(0)
persons_tract_race = persons_tract_race.reset_index().pivot(index='tract',columns="race", values="Unnamed: 0").dropna(0)
persons_tract_hisp = persons_tract_hisp.reset_index().pivot(index='tract',columns="hispanic",values="Unnamed: 0").dropna(0)
persons = [persons_tract_age, persons_tract_sex, persons_tract_race,persons_tract_hisp ]
persons_synth = reduce(lambda left,right: pd.merge(left,right,on="tract"),persons)

## adding prefix to columns
persons_synth.columns = ['s_' + str(col) for col in persons_synth.columns]
persons_synth= persons_synth.rename(columns = {"s_no":"s_nonhispanic","s_yes":"s_hispanic"})

In [93]:
persons_meck .sort_values(by="HISP")

,Unnamed: 0,Unnamed: 0.1,serialno,SPORDER,puma00,puma10,AGEP,JWTR,RELP,SCH,SCHL,SEX,WKHP,ESR,HISP,PERNP,RAC1P,hispanic,person_age,person_sex,race,cat_id,hh_id,state,county,tract,block group
0,0,0,2012000015479,1,-9,3107,62,1.0,0,1.0,16.0,2,44.0,1.0,1,87000.0,1,no,above 60,female,white,138267,186,37,119,005715,1
599483,599483,500,2013000592360,3,-9,3103,18,1.0,2,2.0,18.0,1,15.0,1.0,1,4800.0,2,no,19 and under,male,black,138245,3047,37,119,005524,5
599484,599484,501,2013000592360,2,-9,3103,27,1.0,2,2.0,19.0,2,40.0,1.0,1,17000.0,2,no,20 to 35,female,black,138249,3047,37,119,005524,5
599485,599485,502,2013000760556,1,-9,3103,54,1.0,0,1.0,21.0,2,50.0,1.0,1,57000.0,2,no,35 to 60,female,black,138257,2730,37,119,005524,5
599486,599486,503,2013000760556,1,-9,3103,54,1.0,0,1.0,21.0,2,50.0,1.0,1,57000.0,2,no,35 to 60,female,black,138257,2731,37,119,005524,5
599487,599487,504,2013000760556,1,-9,3103,54,1.0,0,1.0,21.0,2,50.0,1.0,1,57000.0,2,no,35 to 60,female,black,138257,2732,37,119,005524,5
599488,599488,505,2013000760556,1,-9,3103,54,1.0,0,1.0,21.0,2,50.0,1.0,1,57000.0,2,no,35 to 60,female,black,138257,3090,37,119,005524,5
599489,599489,506,2013000760556,2,-9,3103,21,NaN,10,1.0,17.0,1,40.0,6.0,1,21000.0,2,no,20 to 35,male,black,138253,2730,37,119,005524,5
599490,599490,507,2013000760556,2,-9,3103,21,NaN,10,1.0,17.0,1,40.0,6.0,1,21000.0,2,no,20 to 35,male,black,138253,2731,37,119,005524,5
599491,599491,508,2013000760556,2,-9,3103,21,NaN,10,1.0,17.0,1,40.0,6.0,1,21000.0,2,no,20 to 35,male,black,138253,2732,37,119,005524,5


In [98]:
persons_var = persons_meck.drop(columns=['Unnamed: 0.1', 'serialno', 'SPORDER', 'puma00', 'puma10',
       'AGEP', 'JWTR', 'RELP', 'SCH', 'SCHL', 'SEX', 'WKHP', 'ESR', 'HISP',
       'PERNP', 'RAC1P',
       'cat_id', 'hh_id', 'state', 'county', 'block group'])
persons_tract_age = persons_var.groupby(["tract","person_age"]).count()
persons_tract_sex = persons_var.groupby(["tract","person_sex"]).count()
persons_tract_race = persons_var.groupby(["tract","race"]).count()
persons_tract_hisp = persons_var.groupby(["tract","hispanic"]).count()

persons_tract_age = persons_tract_age.reset_index().pivot(index='tract',columns="person_age", values="Unnamed: 0").dropna(0)
persons_tract_sex = persons_tract_sex.reset_index().pivot(index='tract',columns="person_sex", values="Unnamed: 0").dropna(0)
persons_tract_race = persons_tract_race.reset_index().pivot(index='tract',columns="race", values="Unnamed: 0").dropna(0)
persons_tract_hisp = persons_tract_hisp.reset_index().pivot(index='tract',columns="hispanic",values="Unnamed: 0").dropna(0)
persons = [persons_tract_age, persons_tract_sex, persons_tract_race,persons_tract_hisp ]
persons_synth = reduce(lambda left,right: pd.merge(left,right,on="tract"),persons)
persons_synth.columns = ['s_' + str(col) for col in persons_synth.columns]
persons_synth= persons_synth.rename(columns = {"s_no":"s_nonhispanic","s_yes":"s_hispanic"})

In [99]:
persons_synth.head()

,s_19 and under,s_20 to 35,s_35 to 60,s_above 60,s_female,s_male,s_asian,s_black,s_other,s_white,s_nonhispanic,s_hispanic
tract,,,,,,,,,,,,
000100,77,1927,1607,406,2050,1967,76.0,417.0,100.0,3424.0,3631.0,386.0
000300,13,186,326,89,286,328,6.0,189.0,15.0,404.0,607.0,7.0
000400,36,1478,725,177,1138,1278,53.0,440.0,62.0,1861.0,2259.0,157.0
000500,366,2146,991,437,1832,2108,56.0,1050.0,98.0,2736.0,3648.0,292.0
000600,357,1299,731,188,1293,1282,11.0,961.0,44.0,1559.0,2375.0,200.0


In [87]:
persons

[person_age  19 and under  20 to 35  35 to 60  above 60
 tract                                                 
 000100                77      1927      1607       406
 000300                13       186       326        89
 000400                36      1478       725       177
 000500               366      2146       991       437
 000600               357      1299       731       188
 000700                94       264       251        71
 000800               876       608       673       218
 000900               291       385       560       266
 001000               445       811       911       285
 001100               409       618       858       189
 001200              1019      1196      1977       658
 001300               588       807      1221       691
 001400               371       847       991       189
 001504              2096      1255      1918       950
 001505               832       679      1118       567
 001507              1186       930      1084   

In [ ]:
cars_output.to_csv("data_outputs/20190323_compiled/37119_cars_grouped_tract.csv")
population.to_csv("data_outputs/20190323_compiled/37119_population_grouped_tract.csv")
# leaving off here: concatenate dfs based on tract number to reduce # joins in GIS
# join in gis then compute total aggregate error + look at leah's comments again

In [78]:
ACS_pop = pd.read_csv("data_outputs/20190326_census_aggregates/37119_people_meck.csv", converters={'tract': lambda x: str(x)})

In [101]:
people_forgis = ACS_pop.merge(population,how='outer', on="tract")

In [104]:
# make geoid
people_forgis['geoid'] = '37119' + people_forgis['tract'].astype(str)

In [107]:
people_forgis.head()

,tract,19 and under,20 to 35,35 to 60,above 60,total,asian,black,other,white,female,male,pop_tot,geoid
0,000100,244,2713,1544,430,4931,289,501,329,3812,1989,2942,4017.0,37119000100
1,000300,18,204,340,83,645,24,150,28,443,304,341,614.0,37119000300
2,000400,52,1751,661,176,2640,113,407,104,2016,1160,1480,2416.0,37119000400
3,000500,834,2791,981,376,4982,149,1278,332,3223,2167,2815,3940.0,37119000500
4,000600,409,1588,671,204,2872,9,1140,119,1604,1501,1371,2575.0,37119000600


In [111]:
# absolute error
people_forgis['pop_error'] = people_forgis['pop_tot']-people_forgis['total']

In [113]:
#percent error by tract
people_forgis['pop_error_perc'] = people_forgis['pop_error']/people_forgis['total']

In [116]:
people_forgis.sort_values(by='pop_error_perc')

,tract,19 and under,20 to 35,35 to 60,above 60,total,asian,black,other,white,female,male,pop_tot,geoid,pop_error,pop_error_perc
130,005604,3018,2203,263,51,5535,519,1436,296,3284,2623,2912,1315.0,37119005604,-4220.0,-0.762421
99,004700,799,462,454,184,1899,23,1440,57,379,1003,896,926.0,37119004700,-973.0,-0.512375
211,006109,962,1571,1472,604,4609,0,3747,288,574,1846,2763,2691.0,37119006109,-1918.0,-0.416142
225,006403,1938,1425,1540,973,5876,118,807,289,4662,3093,2783,3903.0,37119006403,-1973.0,-0.335773
132,005609,806,2677,347,54,3884,465,1688,388,1343,2035,1849,2731.0,37119005609,-1153.0,-0.296859
104,005200,705,543,652,364,2264,0,1882,74,308,1127,1137,1711.0,37119005200,-553.0,-0.244258
80,003600,1199,1222,1136,584,4141,59,2702,264,1116,2137,2004,3236.0,37119003600,-905.0,-0.218546
3,000500,834,2791,981,376,4982,149,1278,332,3223,2167,2815,3940.0,37119000500,-1042.0,-0.209153
105,005301,1630,1146,1338,325,4439,172,2524,802,941,2360,2079,3529.0,37119005301,-910.0,-0.205001
91,004200,796,641,1318,520,3275,90,2602,68,515,1720,1555,2655.0,37119004200,-620.0,-0.189313


In [118]:
people_forgis.to_csv("data_outputs/20190326_forgis_join/population_acs_synth.csv")